In [6]:
import sys
import gi
import logging

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

uri_name='rtsp://127.0.0.1:8554/test'


def on_pad_added(element, element_src_pad, data):
    #https://github.com/1095788063/deepstream-python-rtsp-video-h264-gstreamer/blob/main/deepstream_rtsp_h264.py
    print("In cb_newpad\n")
    caps = element_src_pad.get_current_caps()
    str = caps.get_structure(0)
    name = str.get_name()
    depay_elem = data

    media = str.get_string("media")
    is_video = media == 'video'
    if 'x-rtp' in name and is_video is True:
        sinkpad = depay_elem.get_static_pad("sink")
        
        state = element_src_pad.link(sinkpad)
        if state != Gst.PadLinkReturn.OK:
            print('source-depay bağlandı')
        else:
            print('Bağlantı sağlanamadı')
    else:
        print('get_name=', name, ' , media=', media)


logging.basicConfig(level=logging.DEBUG, format="[%(name)s] [%(levelname)8s] - %(message)s")
logger = logging.getLogger(__name__)

Gst.init(None)

pipeline=Gst.Pipeline.new('test-pipeline')

source=Gst.ElementFactory.make('rtspsrc','source')
source.set_property('location', uri_name)

depay=Gst.ElementFactory.make("rtph264depay", "depay")

source.connect('pad-added', on_pad_added, depay)


decoder=Gst.ElementFactory.make('avdec_h264','decoder')
sink=Gst.ElementFactory.make('xvimagesink','sink')


pipeline.add(source)
pipeline.add(decoder)
pipeline.add(sink)


if depay.link(decoder):
    logger.info('videoconvert-x264enc Bağlandı')
if decoder.link(sink):
    logger.info('x264enc-sink Bağlandı')

ret=pipeline.set_state(Gst.State.PLAYING)

if ret==Gst.StateChangeReturn.FAILURE:
    logger.error("pipeline'da sıkıntı var")
    sys.exit(1)

bus = pipeline.get_bus()
msg = bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE, 
    Gst.MessageType.ERROR | Gst.MessageType.EOS)

if msg:
    if msg.type == Gst.MessageType.ERROR:
        err, debug_info = msg.parse_error()
        logger.error(f"Error received from element {msg.src.get_name()}: {err.message}")
        logger.error(f"Debugging information: {debug_info if debug_info else 'none'}")

    elif msg.type == Gst.MessageType.EOS:
        logger.info("End-Of-Stream reached.")

    else:
        # This should not happen as we only asked for ERRORs and EOS
        logger.error("Unexpected message received.")

pipeline.set_state(Gst.State.NULL)


(python3:13092): GStreamer-WARNING **: 11:32:01.818: Trying to link elements depay and decoder that don't share a common ancestor: depay hasn't been added to a bin or pipeline, but decoder is in test-pipeline

(python3:13092): GStreamer-WARNING **: 11:32:01.818: Trying to link elements depay and decoder that don't share a common ancestor: depay hasn't been added to a bin or pipeline, but decoder is in test-pipeline
[__main__] [    INFO] - x264enc-sink Bağlandı
[__main__] [   ERROR] - Error received from element source: Could not open resource for reading.
[__main__] [   ERROR] - Debugging information: gstrtspsrc.c(6232): gst_rtspsrc_setup_auth (): /GstPipeline:test-pipeline/GstRTSPSrc:source:
No supported authentication protocol was found


<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>